In [14]:
teams = [
 '1Duke',
 '4VT',
 '3LSU',
 '2MSU',
 '1Zaga',
 '4FSU',
 '3TT',
 '2MICH',
 '1UVA',
 '12Oregon',
 '3Purdue',
 '2TENN',
 '1UNC',
 '5Auburn',
 '3HOU',
 '2Kentucky',
 'out',
]

mikes_picks = [
    [
     '1Duke',
     '2MSU',
     'out',
     '2MICH',
     '1UVA',
     'out',
     '1UNC',
     '3HOU',
    ],[
     '1Duke',
     '2MICH',
     '1UVA',
     '1UNC',
    ],[
     '1Duke',
     '1UVA',
    ],[
     '1UVA'
    ]
]

brads_picks = [
    [
     '1Duke',
     '2MSU',
     '1Zaga',
     '2MICH',
     '1UVA',
     '3Purdue',
     '1UNC',
     '2Kentucky',
    ],[
     '2MSU',
     '1Zaga',
     '1UVA',
     '1UNC',
    ],[
     '2MSU',
     '1UNC',
    ],[
     '1UNC'
    ]
]

erics_picks = [
    [
     '1Duke',
     '2MSU',
     '1Zaga',
     'out',
     '1UVA',
     '3Purdue',
     '5Auburn',
     '3HOU',
    ],[
     '2MSU',
     '1Zaga',
     '1UVA',
     '3HOU'
    ],[
     '2MSU',
     '1UVA',
    ],[
     '1UVA'
    ]
]

likelihoods = {
 '1Duke': [75,55,33,20],
 '4VT': [25,14,6,2],
 '3LSU': [26,5,2,.5],
 '2MSU': [74,26,13,7],
 '1Zaga': [74,48,25,14],
 '4FSU': [26,12,5,2],
 '3TT': [51,20,8,4],
 '2MICH': [49,20,9,5],
 '1UVA': [87,53,33,18],
 '12Oregon': [13,3,1,0.3],
 '3Purdue': [49,21,11,5],
 '2TENN': [51,23,12,5],
 '1UNC': [62,38,18,9],
 '5Auburn': [38,20,8,3],
 '3HOU': [44,17,6,2],
 '2Kentucky': [56,25,10,4],
 'out': [0,0,0,0],    
}
values = [40, 80, 160, 320]

In [24]:
def expectedPointsFromAllRounds(picks, likelihoods, values):
    points = 0
    for i, roundd in enumerate(picks):
        for pick in roundd:
            points += values[i]*likelihoods[pick][i]/100.0
    return points
        
    

In [26]:
print(expectedPointsFromAllRounds(mikes_picks, likelihoods, values))
print(expectedPointsFromAllRounds(brads_picks, likelihoods, values))
print(expectedPointsFromAllRounds(erics_picks, likelihoods, values))

452.40000000000003
420.8
422.80000000000007


In [52]:
'''
given an even number of teams, matches up 0vs1, 2vs3, etc
each member of the array is the team name and their chance of winning in this round
given they make it. calculated by chance of winning in the round / chance of making the round

output is a list of outcomes with likelihood of that list
ex for first round
we have 
duke vt
lsu msu
zaga fsu
[
75 (duke)
25 (vt)
]
then
[
19 (duke, lsu)
56 (duke, msu)
6   (vt, lsu)
19   (vt, msu)
]

'''

'''
we don't have pair wise match-up percentages, but we can approximate
we know duke right now has a 20% of winning the finals and a 33% of making the finals
which means on average they have a 20/33=61% of winning the finals if they make it
but if oregon is their opponent, that is higher
oregon has a .3 /1=30% chance of winning the finals if they make it
so we can bump both of those up to 61/91 and 30/91. 
this isn't perfect, in fact it makes it seem like oregon would rather have
duke than an average opponent, which doesn't make sense
and it doesn't really bump dukes chances that much if oregon is their opponent,
but this isn't a major effect and is fine for now
'''
def scenario_for_first_team(initial_scenario_likelihood, winners, rnd_teams, next_rnd_teams, team_a, team_a_winning_chance, team_b_winning_chance):
    chance_team_a_wins = team_a_winning_chance / (team_a_winning_chance+team_b_winning_chance)
    new_scenario = (initial_scenario_likelihood*chance_team_a_wins, winners+[team_a], rnd_teams, next_rnd_teams+[team_a])
    return new_scenario

def add_game(scenarios, team_a, team_a_winning_chance, team_b, team_b_winning_chance):
    new_scenarios = []
    for (likelihood, winners, rnd_teams, next_rnd_teams) in scenarios:
        new_scenarios.append(scenario_for_first_team(likelihood, winners, rnd_teams, next_rnd_teams, team_a, team_a_winning_chance, team_b_winning_chance))
        new_scenarios.append(scenario_for_first_team(likelihood, winners, rnd_teams, next_rnd_teams, team_b, team_b_winning_chance, team_a_winning_chance))
    return new_scenarios

def get_winning_chance(team, rnd, likelihoods):
    denom = 100 if rnd==0 else likelihoods[team][rnd-1]
    return likelihoods[team][rnd] / denom
'''
game_list
duke
uva
likelihoods
duke 70
uva 30

process games returns
scenarios =
(70, [duke])
(30, [uva])


game_list
duke
mich
uva
unc
likelihood
duke 90 50
mich 10 5
uva 80 30
unc 20 15

scenarios
[]
(90, [duke], [duke])
(10, [mich], [duke])
'''

flatten = lambda outer_list: [item for sublist in outer_list for item in sublist]



def process_scenarios(scenarios_for_round, rnd, likelihoods):
    rnd_teams = scenarios_for_round[0][2]
    while rnd_teams:
        team_a = rnd_teams.pop(0)
        team_b = rnd_teams.pop(0)
        team_a_winning_chance = get_winning_chance(team_a, rnd, likelihoods)
        team_b_winning_chance = get_winning_chance(team_b, rnd, likelihoods)
        scenarios_for_round = add_game(scenarios_for_round, team_a, team_a_winning_chance, team_b, team_b_winning_chance)
    # now we have scenarios for round
    updated_scenarios = []
    for (likelihood, winners, rnd_teams, next_rnd_teams) in scenarios_for_round:
        new_item = (likelihood, winners, next_rnd_teams, [])
        updated_scenarios.append(new_item)
    
    sample = updated_scenarios[0]
    sample_next_rnd_teams = sample[2]
    if len(sample_next_rnd_teams) == 1:
        return updated_scenarios
    else:
        return process_scenarios(updated_scenarios, rnd+1, likelihoods)
    
def create_scenarios():
    likelihoods = {
     '1Duke': [75,55,33,20],
     '4VT': [25,14,6,2],
     '3LSU': [26,5,2,.5],
     '2MSU': [74,26,13,7],
     '1Zaga': [74,48,25,14],
     '4FSU': [26,12,5,2],
     '3TT': [51,20,8,4],
     '2MICH': [49,20,9,5],
     '1UVA': [87,53,33,18],
     '12Oregon': [13,3,1,0.3],
     '3Purdue': [49,21,11,5],
     '2TENN': [51,23,12,5],
     '1UNC': [62,38,18,9],
     '5Auburn': [38,20,8,3],
     '3HOU': [44,17,6,2],
     '2Kentucky': [56,25,10,4],
     'out': [0,0,0,0],    
    }    

    initial_rnd_teams = [
     '1Duke',
     '4VT',
     '3LSU',
     '2MSU',
     '1Zaga',
     '4FSU',
     '3TT',
     '2MICH',
     '1UVA',
     '12Oregon',
     '3Purdue',
     '2TENN',
     '1UNC',
     '5Auburn',
     '3HOU',
     '2Kentucky',
    ]

    # likelihood, winners, rnd_teams, next_rnd_teams
    initial_scenario = (100, [], initial_rnd_teams, [])
    all_scenarios = process_scenarios([initial_scenario], rnd=0, likelihoods=likelihoods)

    return all_scenarios


            


In [119]:
def score(scenario, picks, expanded_values):
    # picks: 
    '''
    mikes_picks = [
    [
     '1Duke',
     '2MSU',
     'out',
     '2MICH',
     '1UVA',
     'out',
     '1UNC',
     '3HOU',
    ],[
     '1Duke',
     '2MICH',
     '1UVA',
     '1UNC',
    ],[
     '1Duke',
     '1UVA',
    ],[
     '1UVA'
    ]
    ----
    scenario
     (0.024345421774373627,
     ['1Duke',
      '3LSU',
      '1Zaga',
      '3TT',
      '1UVA',
      '3Purdue',
      '1UNC',
      '3HOU',
      '1Duke',
      '1Zaga',
      '1UVA',
      '1UNC',
      '1Duke',
      '1UVA',
      '1Duke'],
     ['1Duke'],
     [])
     ----
     expanded values
     [40 40 40 40 40 40 40 40 40 80 80 80 80 160 160 320]
    '''
    flat_picks = flatten(picks)
    winners = scenario[1]

    score = 0
    for i in range(len(winners)):
        if winners[i]==flat_picks[i]:
            score += expanded_values[i]
    return score
        

In [130]:
# brackets:
'''
{
 "mike": 
     [    [
     '1Duke',
     '2MSU',
     'out',
     '2MICH',
     '1UVA',
     'out',
     '1UNC',
     '3HOU',
    ],[
     '1Duke',
     '2MICH',
     '1UVA',
     '1UNC',
    ],[
     '1Duke',
     '1UVA',
    ],[
     '1UVA'
    ]],
 "eric": [...]
}
'''
from collections import Counter
# using a counter makes arg max semantic and allows for easy addition of second place counts and so on
# most common is analagous to high scorer, could be built on my own, but this is a little simpler
def accumulate_scenarios(brackets, scenarios, expanded_values):
    wins = Counter()
    seconds = Counter()
    thirds = Counter()

    for scenario in scenarios:
        scenario_scores = Counter()
        for participant in brackets:
            scenario_scores[participant] += brackets[participant][1]
            scenario_scores[participant] += score(scenario, brackets[participant][0], expanded_values)
        (winner, s1), (second, s2), (third, s3) = scenario_scores.most_common(3)
        prevalence = scenario[0]
        wins[winner] += prevalence
        seconds[second] += prevalence
        thirds[third] += prevalence
    return wins, seconds, thirds
        

        

In [131]:
brackets = {
    "mike": (mikes_picks, 450),
    "brad": (brads_picks, 570),
    "eric": (erics_picks, 470)
}
expanded_values = [40]*8+[80]*4+[160]*2+[320]
all_scenarios = create_scenarios()

wins, seconds, thirds = accumulate_scenarios(brackets, all_scenarios, expanded_values)

In [132]:
wins.most_common()

[('mike', 52.40613345568731),
 ('brad', 27.99663694249405),
 ('eric', 19.59722960181833)]

In [133]:
seconds.most_common()

[('eric', 40.546660519124316),
 ('mike', 35.10047148739985),
 ('brad', 24.35286799347578)]

In [134]:
thirds.most_common()

[('brad', 47.65049506402995),
 ('eric', 39.85610987905733),
 ('mike', 12.493395056912616)]